<a href="https://colab.research.google.com/github/RitinDev/projects-programming-data-sciences/blob/main/class3/RitinMalhotra_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 | Ritin Malhotra | rm5486
Working with NewsAPI and IBM Watson NLP Tool(s) to create a database of popular articles related to the passing of Queen Elizabeth II, and the sentiments associated with each article. 



## Workflow
1. Initialize the sentiment and emotion analysis function through IBM Watson. Since I ran into payment issues for the IBM Watson API due to not owning a U.S. based credit card, I am sharing the API key with a friend. While we will, in all probability, not run out of the daily quota for sentiment analysis, I cannot give any guarantees. If need be, the `URL` and `API_KEY` in the second code block can be changed. Apolgoies for the inconvenience.

2. Get the top headlines related to Queen Elizabeth II using NewsAPI. I have decided to use `get_top_headlines()` instead of `get_everything()` since I do not want to add too many articles to the tables and analysis.

3. Create a database and add the `content` and `URL` for each article to a single table. Print this table out.

4. Run sentiment analysis on each article using its `URL`. Add the sentiment analysis scores + URL to a list of dictionaries, which will enable us to easily add the values to another table. I have used Python's exception handling to ensure that a single bad article does not affect the flow of the program.

5. Add `sentiment_score` and `url` to a new table. Print this table out.

6. Drop both the tables so that new ones are created whenever the program is run.



In [115]:
import requests

In [116]:
'''URL = 'https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/9e683088-0d12-4399-8118-518f3e60e8c4'

# My own API key. It may run out of quota
# You can register and get your own credentials
# The ones below have a quota of 1000 calls per day 
# and can run out quickly if multiple people use these
API_KEY = 'yx39wyiwPNGm7DoDUPCSJB4SzFkr0qurARfbGYyEdaoC'''

URL = 'https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/d56bc7f9-88a8-4b7a-aa86-0557ca745925'
API_KEY = 'xUQrt-jdAvneYY4cC7oAhLzCkS-RrDldfPfarjn9kBwl'

def analyzeText(text=None, url=None):

    endpoint = f"{URL}/v1/analyze"
    username = "apikey"
    password = API_KEY
    
    parameters = {
        'features': 'emotion,sentiment',
        'version' : '2022-04-07',
        'text': text,
        'language' : 'en',
        'url' : url # this is an alternative to sending the text
    }

    resp = requests.get(endpoint, params=parameters, auth=(username, password))
    
    return resp.json()

In [117]:
# !sudo -H pip3 install newsapi-python
!pip install newsapi-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [118]:
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='a7eab21c34e545dba418c4344d59a54f')

# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(q = 'queen',
                                          language='en',
                                          page_size=100)

# top_headlines.keys()
articles = top_headlines['articles']

'''for article in articles:
    article_url = article['url']
    # print(article)
    print(article['content'])
    print(article['url'])
    data = analyzeText(url=article_url)
    print(data['sentiment']['document'])
    print(data['emotion']['document'])'''

"for article in articles:\n    article_url = article['url']\n    # print(article)\n    print(article['content'])\n    print(article['url'])\n    data = analyzeText(url=article_url)\n    print(data['sentiment']['document'])\n    print(data['emotion']['document'])"

In [119]:
!sudo pip3 install -U -q PyMySQL sqlalchemy sql_magic

In [120]:
from sqlalchemy import create_engine

conn_string = "mysql+pymysql://{user}:{password}@{host}/".format(
    host="db.ipeirotis.org", user="student", password="dwdstudent2015"
)

engine = create_engine(conn_string)

In [121]:
# Query to create a database
db_name = "public"
create_db_query = (
    f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8'"
)

# Create a database
engine.execute(create_db_query)

In [122]:
suffix = "rm5486"
table_name = f"{suffix}_news"
# Create a table
create_table_query = f"""CREATE TABLE IF NOT EXISTS {db_name}.{table_name} 
                                (content varchar(1000), 
                                url varchar(500), 
                                PRIMARY KEY(url)
                                )"""
engine.execute(create_table_query)

In [123]:
query_template = f"""
                    INSERT IGNORE INTO 
                    {db_name}.{table_name}(content,  url) 
                    VALUES (%s, %s)
                  """
for article in articles:
    content = article['content']
    url = article['url']

    # print("Inserting article", content, "with", url, "as URL")
    query_parameters = (content, url)
    engine.execute(query_template, query_parameters)

In [124]:
results = engine.execute(f"SELECT * FROM {db_name}.{table_name}")
rows = results.fetchall()
results.close()

In [125]:
for row in rows:
    print("Content:", row["content"])
    print("URL:", row["url"])
    print("=============================================")

Content: The Chinese vice-president, Wang Qishan, is to attend the Queens funeral in a move that has prompted complaints from a group of British Conservative MPs that have been banned from travelling to China… [+3987 chars]
URL: https://amp.theguardian.com/uk-news/2022/sep/15/anger-among-mps-as-chinese-vice-president-to-attend-queens-funeral
Content: Workers are scrambling to find last-minute child care in several provinces after governments announced the sudden closure of schools to mourn Queen Elizabeth.
The four Atlantic provinces, British Co… [+3083 chars]
URL: https://globalnews.ca/news/9130909/canada-school-closures-sept-19-queen-funeral/
Content: The passing of the Queen, a unifying figure more beloved than her son, King Charles III, comes as several Commonwealth realms are reassessing their relationships with the crown 
Author of the articl… [+11403 chars]
URL: https://nationalpost.com/news/as-elizabeth-gives-way-to-charles-realms-consider-severing-ties
Content: The new Prince 

In [126]:
sentiment_scores = []

for row in rows:
  article_url = row["url"]
  data = analyzeText(url=article_url)
  entry = {}
  entry['url'] = article_url
  try:
    entry['score'] = data['sentiment']['document']['score']
    # print(data['sentiment']['document']['score'])
  except KeyError:
    entry['score'] = 'ERROR: Cannot determine sentiment analysis score for this article'
    # print("ERROR: Sentiment not detected")
  
  # print(entry)
  sentiment_scores.append(entry);

In [127]:
table_name = f"{suffix}_sentiment_score"
# Create a table
create_table_query = f"""CREATE TABLE IF NOT EXISTS {db_name}.{table_name} 
                                (url varchar(255),
                                sentiment_score varchar(255), 
                                PRIMARY KEY(url)
                                )"""
engine.execute(create_table_query)

In [128]:
query_template = f"""
                    INSERT IGNORE INTO 
                    {db_name}.{table_name}(url, sentiment_score) 
                    VALUES (%s, %s)
                  """
for score in sentiment_scores:
    url = score['url']
    sentiment_score = score['score']

    # print("Inserting URL", url, "with", sentiment_score, "as score")
    query_parameters = (url, sentiment_score)
    engine.execute(query_template, query_parameters)

In [129]:
results = engine.execute(f"SELECT * FROM {db_name}.{table_name}")
rows = results.fetchall()
results.close()

In [130]:
for row in rows:
    print("URL:", row["url"])
    print("Sentiment Score:", row["sentiment_score"])
    print("=============================================")

URL: https://amp.theguardian.com/uk-news/2022/sep/15/anger-among-mps-as-chinese-vice-president-to-attend-queens-funeral
Sentiment Score: 0.41797
URL: https://globalnews.ca/news/9130909/canada-school-closures-sept-19-queen-funeral/
Sentiment Score: -0.74832
URL: https://nationalpost.com/news/as-elizabeth-gives-way-to-charles-realms-consider-severing-ties
Sentiment Score: 0.258092
URL: https://news.sky.com/story/prince-and-princess-of-wales-view-tributes-to-queen-at-sandringham-12697938
Sentiment Score: -0.366433
URL: https://nz.news.yahoo.com/hidden-detail-in-photo-of-kate-at-queens-coffin-procession-225141994.html
Sentiment Score: -0.463513
URL: https://theindependent.sg/he-predicted-queen-elizabeth-iis-death-and-now-he-predicts-king-charless-death/
Sentiment Score: -0.467971
URL: https://www.9news.com.au/national/australia-breaking-news-today-live-queen-elizabeth-ii-king-charles-iii-updates-latest-headlines/92e62e8a-fd45-4cea-b00a-91b8eeaeee0c
Sentiment Score: 0.665348
URL: https://ww

In [131]:
drop_table_query = f"DROP TABLE IF EXISTS {db_name}.{table_name}"
engine.execute(drop_table_query)

table_name = f"{suffix}_news"

drop_table_query = f"DROP TABLE IF EXISTS {db_name}.{table_name}"
engine.execute(drop_table_query)

print("Dropped table from database.")
print("A new one will be created everytime we run this program")

Dropped table from database.
A new one will be created everytime we run this program
